In [ ]:
from src import config
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]=config.NUM_DEVICE
DEVICE='cuda'

import cv2
import sys
import numpy as np

import optuna
from pathlib import Path
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
import catalyst
from catalyst import dl
from catalyst import metrics

from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

import math
import pandas as pd
import torch.nn.functional as F



In [ ]:
from src.architecture import LumbarHmModel
from src.preprocessing import LumbarDataset, get_aug_test, randAugment

In [ ]:
data = pd.read_csv(config.PATH_TO_DATA)
train, test = train_test_split(data, test_size=0.2, random_state=777)

In [ ]:
train_dataset = LumbarDataset(train, tfms=get_aug_test())
dataloader_train = DataLoader(train_dataset, batch_size=16, shuffle=True)

test_dataset = LumbarDataset(test, tfms=get_aug_test())
dataloader_val = DataLoader(test_dataset, batch_size=1, shuffle=False)

In [ ]:
model = LumbarHmModel("efficientnet-b2", "Unet")
criterion = nn.MSELoss()
optimizer = torch.optim.AdamW(model.parameters())

In [ ]:
loaders = {
    "train": dataloader_train,
    "valid": dataloader_val,
}

In [ ]:
runner = dl.SupervisedRunner(
    input_key="features", output_key="logits", target_key="targets", loss_key="loss"
)

In [ ]:
runner.train(
      model=model,
      criterion=criterion,
      optimizer=optimizer, 
      loaders=loaders, 
      num_epochs=300,
      verbose=True, # you can pass True for more precise training process logging
      logdir="/storage1/ryazantsev/lumbar_spine/experiments/unet+efficientnetb2",
      timeit=False,
      valid_loader="valid",
      valid_metric="loss",
      minimize_valid_metric=True,
        callbacks=[
            dl.EarlyStoppingCallback(patience=25, loader_key='valid', metric_key="loss", minimize=True),
        ],
    )